In this model we are using Random Forest Classifier to prepare model.
Dataset is loaded from google drive using the code below.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive'

/content/drive/MyDrive


In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestClassifier as rfc
from sklearn import metrics

tf.io.gfile.listdir() is a tensorflow function used to list the files present in given directory

Syntax:

  tf.io.gfile.listdir(str:filepath)

In [ ]:
gl=tf.io.gfile.listdir("/content/drive/MyDrive/yoga_pose/Train/goddess")
pl=tf.io.gfile.listdir("/content/drive/MyDrive/yoga_pose/Train/plank")
tl=tf.io.gfile.listdir("/content/drive/MyDrive/yoga_pose/Train/tree")
wl=tf.io.gfile.listdir("/content/drive/MyDrive/yoga_pose/Train/warrior2")
dl=tf.io.gfile.listdir("/content/drive/MyDrive/yoga_pose/Train/downdog")

Bellow operation is performed to copy all ".jpg" files from yoga_pose to yTrain

In [ ]:
c=[]
l=[dl,gl,pl,tl,wl]
n=['downdog','goddess','plank','tree','warrior2']
for j in l:
  for i in j:
  if(i.endswith(".jpg")):
    src="/content/drive/MyDrive/yoga_pose/Train/"+j+"/"+i
    des="/content/drive/MyDrive/yTrain/warrior2/"+j+"/"+i
    tf.io.gfile.copy(src,des)



list is created by name "train_label" to store the label of training images 

In [ ]:
train_label=[]
for i in dl:
  train_label.append("downdog")

In [ ]:
for i in gl:
  train_label.append("goddess")

In [ ]:
for i in pl:
  train_label.append("plank")

In [41]:
for i in tl:
  train_label.append("tree")

In [42]:
for i in wl:
  train_label.append("warrior2")

to improve the quality of jpg files I have used tf.image.adjust_jpeg_quality() function 

syntax:

tf.image.adjust_jpeg_quality(image object, quality)

In [ ]:
di=tf.io.gfile.listdir("/content/drive/MyDrive/yTrain/downdog")
gi=tf.io.gfile.listdir("/content/drive/MyDrive/yTrain/goddess")
pi=tf.io.gfile.listdir("/content/drive/MyDrive/yTrain/plank")
ti=tf.io.gfile.listdir("/content/drive/MyDrive/yTrain/tree")
wi=tf.io.gfile.listdir("/content/drive/MyDrive/yTrain/warrior2")
l=[di,gi,pi]
n=['downdog','goddess','plank']
for i in range(3):
  for j in l[i]:
    path="/content/drive/MyDrive/yTrain/"+n[i]+"/"+j
    im=cv.imread(path)
    tf.image.adjust_jpeg_quality(im,50)

In this model train images are not shuffled because Random forest model doesn't require shuffeling. If these images are shuffled then train_label list will provide wrong labelling.

actuall there are only 5 classes but due to repeated changes in traning dataset it displayes 6 where 6th class is check_points

In [40]:
path="/content/drive/MyDrive/yTrain"
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    path,
    batch_size=1,
    shuffle=False
)

Found 920 files belonging to 6 classes.


In [ ]:
test_labels=[]
dir=tf.io.gfile.listdir("/content/drive/MyDrive/yoga_pose/Test")
for s in dir:
  sl=tf.io.gfile.listdir("/content/drive/MyDrive/yoga_pose/Test/"+s)
  for i in sl:
    test_labels.append(s)

In [ ]:
tpath="/content/drive/MyDrive/yoga_pose/Test"
test_ds=tf.keras.preprocessing.image_dataset_from_directory(
    tpath,
    batch_size=1,
    shuffle=False
)

Found 470 files belonging to 5 classes.


In the code below we are defining feature_extractor

In [ ]:
num_classes = 5
from tensorflow.keras import layers

feature_extractor = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(256, 256, 3)),
  layers.Conv2D(16, 3, padding='same', activation='sigmoid'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='sigmoid'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='sigmoid'),
  layers.MaxPooling2D(),
  layers.Flatten()
])


below code is used to provide output using Dense layers

In [ ]:
from tensorflow.keras.layers import Dense
f_op=feature_extractor.output
f_op=Dense(128,activation='sigmoid')(f_op)
op_layer=Dense(5,activation='softmax')(f_op)

I have used preprocessing module to convert labels in string format to int using LabelEncoder(). One hot encoding is not used in Random Forest Model.

In [ ]:
from sklearn import preprocessing
en=preprocessing.LabelEncoder()
en.fit(train_label)
en.fit(test_labels)
train_en_label=en.transform(train_label)
test_en_label=en.transform(test_labels)

Below code was used to create the final Random Forest Model. by taking data from feature_extractor as input and encoded train labels as output.

In [ ]:
train_rfc=feature_extractor.predict(train_ds)
rf_model=rfc(n_estimators=100,random_state=42)
rf_model.fit(train_rfc,train_en_label)

ValueError: ignored

Same process is repeated with test data.

In [ ]:
test_rfc=feature_extractor.predict(test_ds)
t_model=rfc(n_estimators=100,random_state=42)
t_model.fit(test_rfc,test_en_label)


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)